# Introduction to machine learning 2022 term project

In [1]:
# imports
import pandas as pd
import numpy as np

# sklearn models
from sklearn.model_selection import cross_validate,cross_val_score, train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,BayesianRidge
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier


#Feature selection imports
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
npf = pd.read_csv("npf_train.csv")
test = pd.read_csv("npf_test_hidden.csv")

### Data preprocessing

Column "class2" added that is "event" if "class4" is Ib, Ia or II, otherwise labeled as "nonevent". 

In [3]:
class2 = np.array(["nonevent", "event"])
npf["class2"] = class2[(npf["class4"]!="nonevent").astype(int)]


In [4]:
npf.head(n=2)
train, test = train_test_split(npf,test_size=0.33, random_state=42)
print("Train size:", len(train),"Test size:", len(test))

Train size: 310 Test size: 154


In [5]:
#X = npf[labels]
X = train.filter(regex=".mean")
y = train["class2"]
X_test = test.filter(regex=".mean")
y_test = test["class2"]

In [6]:
Classifier_Names = ["dummyRegressor",
                    "logisticRegressor",
                    "GaussianNaiveBays",
                    "SVC",
                    "SVC-sigmoid",
                    "SVC-poly",
                    "RandomForest",
                    "AdaBoost",
                    "NearestNeighbour",
                    "GaussianProcess",
                    "QDA",
                    "DecisionTree"]


classifiers = [DummyClassifier(),
          LogisticRegression(solver="saga"),
          GaussianNB(),
          SVC(probability=True),
          SVC(kernel="sigmoid",probability=True),
          SVC(kernel="poly",probability=True),
          RandomForestClassifier(),
          AdaBoostClassifier(),
          KNeighborsClassifier(3),
          GaussianProcessClassifier(1.0 * RBF(1.0)),
          QuadraticDiscriminantAnalysis(),
          DecisionTreeClassifier(max_depth=5)
          ]
models = [Pipeline([("feature_selection",SelectKBest(f_classif, k = 10)),("classifier",c)]) for c in classifiers]
res = pd.DataFrame(index=Classifier_Names)

In [7]:
perplexity = lambda p: np.exp(-np.mean(np.log((test["class2"]!="nonevent").astype(int)*p + (1 - (test["class2"]!="nonevent").astype(int)) * (1 - p))))

In [8]:
res["train"] = [m.fit(X,y).score(X,y) for m in models]
res["cv"] = [cross_val_score(m, X, y, cv=10).mean() for m in models]
res["test"] = [m.fit(X,y).score(X_test,y_test) for m in models]
res["perplexity"] = [perplexity(m.fit(X,y).predict_proba(X_test)[:,1]) for m in models]

/usr/lib/python3/dist-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "
/usr/lib/python3/dist-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/usr/lib/python3/dist-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "
/usr/lib/python3/dist-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The default value of strategy will change from "
/usr/lib/python3/dist-packages/sklearn/dummy.py:131: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  warnings.warn("The 

In [9]:
res

,train,cv,test,perplexity
dummyRegressor,0.516129,0.474194,0.506494,inf
logisticRegressor,0.761290,0.758065,0.798701,4.834991e+00
GaussianNaiveBays,0.787097,0.783871,0.792208,1.156793e+06
SVC,0.767742,0.764516,0.779221,3.932640e+00
SVC-sigmoid,0.732258,0.622581,0.779221,3.644898e+00
SVC-poly,0.751613,0.745161,0.785714,3.926238e+00
RandomForest,1.000000,0.787097,0.818182,inf
AdaBoost,0.929032,0.777419,0.824675,2.229494e+00
NearestNeighbour,0.874194,0.777419,0.759740,inf
GaussianProcess,0.838710,0.787097,0.857143,6.741112e+00
